In [1]:
from typing import Any
from torch import Tensor
from torch.nn import Module
from torch.optim import Optimizer
from torchsystem.aggregate import Aggregate, Loader
from torchsystem.callbacks import Callback

class Classifier(Aggregate):
    def __init__(self, id: Any, model: Module, criterion: Module, optimizer: Optimizer):
        super().__init__(id)        
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer

    def forward(self, input: Tensor) -> Tensor:
        return self.model(input)

    def loss(self, output: Tensor, target: Tensor) -> Tensor:
        return self.criterion(output, target)

    def fit(self, data: Loader, callback: Callback):
        for batch, (input, target) in enumerate(data, start=1):
            output = self(input)
            loss = self.loss(output, target)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            callback(batch, loss.item(), output, target, input)

    def evaluate(self, data: Loader, callback: Callback):
        for batch, (input, target) in enumerate(data, start=1):
            output = self(input)
            loss = self.loss(output, target)
            callback(batch, loss.item(), output, target, input)

In [2]:
from torch.nn import Module
from torch.nn import ReLU, Linear
from torch.nn import Dropout

class MLP(Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int, p: float):
        super().__init__()
        self.input_layer = Linear(input_size, hidden_size)
        self.activation = ReLU()
        self.dropout = Dropout(p)
        self.output_layer = Linear(hidden_size, output_size)

    def forward(self, sequence: Tensor) -> Tensor:
        sequence = self.input_layer(sequence)
        sequence = self.activation(sequence)
        sequence = self.dropout(sequence)
        sequence = self.output_layer(sequence)
        return sequence

In [3]:
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader

class Digits(Dataset):
    def __init__(self, train: bool, normalize: bool):
        transform = Compose([ToTensor()]) if not normalize else Compose([ToTensor(), Normalize((0.5,), (0.5,))])
        self.dataset = MNIST(root='data/datasets', train=train, download=True, transform=transform)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index: int):
        return self.dataset[index]

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torchsystem.compiler import Compiler
from torchsystem.callbacks import Callbacks, Loss, Accuracy
from torchsystem.loaders import Loaders
from torchsystem.registry import Models, Criterions, Optimizers, Datasets

Models.register(MLP)
Criterions.register(CrossEntropyLoss)
Optimizers.register(Adam)
Datasets.register(Digits)

model = MLP(784, 128, 10, p=0.2)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
loaders = Loaders()
loaders.add('train', Digits(train=True, normalize=True), batch_size=32, shuffle=True)
loaders.add('test', Digits(train=False, normalize=True), batch_size=32, shuffle=False)

callback = Callbacks([Loss(), Accuracy()])
compiler = Compiler(Classifier)
aggregate = compiler.compile('0', model, criterion, optimizer)

In [5]:
from mlregistry import get_metadata
for phase, loader in loaders:
    print(get_metadata(loader.dataset))

Metadata(type='dataset', hash='c728058dbf80e17ebf0aba2bc09e0dc7', name='Digits', arguments={'train': True, 'normalize': True})
Metadata(type='dataset', hash='c728058dbf80e17ebf0aba2bc09e0dc7', name='Digits', arguments={'train': False, 'normalize': True})


In [6]:
from torchsystem.weights import Weights

weights = Weights(directory='data/weights')
weights.store(model, 'model', '123')

In [9]:
print(get_metadata(aggregate.optimizer))

Metadata(type='optimizer', hash='55b114a06ec82cec384573d24663dafe', name='Adam', arguments={'lr': 0.001})
